# Third attempt at learning

In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()
import os
import random
import math
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from scipy.stats import spearmanr
import pymc3 as pm
import theano
theano.config.compute_test_value = 'raise'
%matplotlib inline
from multiprocessing import Pool, cpu_count

SELECTED_DATA_DIR = "../selected-data/"
MOVIES_FILE = "best_movie_ratings_features_engineered.csv"
USERS_FILE = "users_ratings.csv"

## Read raw data

In [ ]:
movies_raw = pd.read_csv(SELECTED_DATA_DIR + MOVIES_FILE, index_col=0)
movies_raw.rating = movies_raw.rating/10
movies_raw.sample()

In [ ]:
users = pd.read_csv(SELECTED_DATA_DIR + USERS_FILE, index_col=0)
users.rating = users.rating/10
users.sample()

## Reduce data dimension (PCA)

In [ ]:
WANTED_DIM = 20

In [ ]:
pca_df = movies_raw[list(movies_raw.columns[2:])]
pca = PCA(n_components=WANTED_DIM)
pca_df = pd.DataFrame(pca.fit_transform(pca_df))
pca_df.index = movies_raw.index

In [ ]:
movies_raw.shape

Pourcentage of variance in dataset conserveted

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
movies = pd.concat([movies_raw[list(movies_raw.columns[:2])], pd.DataFrame(pca_df)] ,axis=1)

## Actions selection function

In [ ]:
def random_choice(user_features, movies, epoch, s):
    """ random approach to the problem, always exploring"""
    return movies.sample()

In [ ]:
def greedy_choice(user_features, movies, epoch, s):
    """ greedy approach to the problem """
    epsilon = 1 / math.sqrt(epoch+1)
    if random.random() > epsilon: # choose the best
        return best_recommandation(user_features, movies, epoch, s)
    else:
        return movies.sample()

def greedy_choice_no_t(user_features, movies, epsilon=0.5):
    """ greedy approach to the problem """
    if random.random() > epsilon: # choose the best
        return best_recommandation(user_features, movies)
    else:
        return movies.sample()

In [ ]:
def bayes_UCB(user_features, movies, epoch, s):
    # Hyperparameters
    c0 = 10
    d0 = 3
    e0 = 0.01
    f0 = 0.001
    g0 = 0.001
    # function
    I = np.eye(user_features.size)
    ratings = np.zeros(movies.shape[0])
    with pm.Model():
        s = pm.Gamma('s', d0, e0)
        sigma = pm.InverseGamma('sigma', f0, g0)
        theta = pm.MvNormal('theta', mu=0.5, cov=c0 * sigma * I)
        rating = pm.Normal('rating', mu=0, sd=sigma, observed=user_features)

        for i, (title, movie) in tqdm(enumerate(movies.iterrows())): 
            movies_features = get_movie_features(movies)
            # Expected value of outcome
            mu = user_features.dot(movies_features) * (1 - np.exp(-epoch/s))
            # Likelihood (sampling distribution) of observations
            rating.mu = mu
            
            step = pm.Metropolis()
            trace = pm.sample(1000, step=step, njobs=1, progressbar=False)
            ratings[i] = rating.distribution.random()[0]
    return movies[movies.index == movies.index[ratings.argmax()]]

## Learning utilities

In [ ]:
def compute_utility(user_features, movie_features, epoch, s):
    """ Compute utility U based on user preferences and movie preferences """
    res = user_features.dot(movie_features) * (1 - math.exp(-epoch/s))
    return res

def get_movie_features(movie):
    """ selected features from dataframe """
    if isinstance(movie, pd.Series):
        return movie[-WANTED_DIM:]
    elif isinstance(movie, pd.DataFrame):
        return get_movie_features(movie.loc[movie.index[0]])
    else:
        raise TypeError("{} should be a Series or DataFrame".format(movie))
    
def best_recommandation(user_features, movies, epoch, s):
    """ Return the movie with the highest utility """
    utilities = np.zeros(movies.shape[0])
    for i, (title, movie) in enumerate(movies.iterrows()):
        movie_features = get_movie_features(movie)
        utilities[i] = compute_utility(user_features, movie_features, epoch - movie.last_t, s)
    return movies[movies.index == movies.index[utilities.argmax()]]

def all_recommandation(user_features, movies):
    """ Return all movies sorted by utility """
    movies = movies.copy()
    movies['utilities'] = movies.apply(lambda mov: compute_utility(user_features, get_movie_features(mov), 1000), axis=1)
    return movies.sort_values(by="utilities")
        
def iterative_mean(old, new, t):
    """ Compute the new mean """
    #print(old)
    #print(new)
    return ((t-1) / t) * old + (1/t) * new
    
def update_features(user_features, movie_features, rating, t):
    """ update the user preferences """
    return iterative_mean(user_features, movie_features * rating, t+1)

## Simulate with one selector (greedy or random or bayes or ...)

And return scoring

In [ ]:
def reinforcement_learning(user, moviestc, choicef, s, numberSimulation):
    if s<200:
        print("WARNING : s is really small, movies will get often repeated")
    user_features = np.zeros(moviestc.shape[1] - 2)
    movies = moviestc.copy()
    cumregret = [0]
    movies.insert(0, 'last_t', np.ones(movies.shape[0]).astype(np.int64))
    for t in range(numberSimulation):
        recommandation = choicef(user_features, movies, t, s)
        recommandation_features = get_movie_features(recommandation)
        user_rating = user.get_value(recommandation.index[0], "rating")
        user_features = update_features(user_features, recommandation_features, user_rating, t)
        utility = compute_utility(user_features, recommandation_features, t, s)
        cumregret.append(cumregret[-1] + (user_rating - utility ))
        movies.loc[movies.index.isin(recommandation.index),'last_t'] = t
    return cumregret

## Simulate many

In [ ]:
def rl_multiple_users(users, movies, algorithms=[greedy_choice, random_choice], s=1, N=20, N_USER=50):
    def wrapper_rl_one_user(args):
        return reinforcement_learning(*args)
    regrets = []        
    for algo in algorithms:
        regret_algo = []
        args = []
        for i in range(N_USER):
            user = users[users.user.isin(users.user.sample())]
            movies_user = movies[movies.index.isin(user.index)]
            choicef = algo
            args.append([user, movies_user, choicef, s, N])
        with Pool(cpu_count()) as p:
            regret_algo = p.map(wrapper_rl_one_user, args)
        regrets.append(regret_algo)
    regrets = [sum(regret)/len(regret) for regret in regrets]
    return regrets

# Results

### Selection one user randomly

In [ ]:
user = users[users.user.isin(users.user.sample())]
user.shape

In [ ]:
# get only movies that this user rated
movies_user = movies[movies.index.isin(user.index)]
movies_user.shape

In [ ]:
cumregret_greedy = reinforcement_learning(user, movies_user, greedy_choice, 300, 100)
cumregret_random = reinforcement_learning(user, movies_user, random_choice, 300, 100)
print(regret_greedy, regret_random)

## Select multiple users and algo

In [ ]:
#res = rl_multiple_users(users, movies, [greedy_choice, bayes_UCB], N=1, N_USER=1)